# Project EShop

This is a project in which we'll put our newly learned database skills to use. The set up fairly simple: Start the project, make a products app, run migrations and done!

In [ ]:
django-admin startproject EShop
cd EShop
python manage.py startapp products
python manage.py migrate

Then we move on to working with Template layer, make some `HTML` files in the templates directory in products app.

Along with some CSS files which are not the main target of this lesson.

Make the views and set up the urls and finally run the server and the set up is complete.

In [ ]:
# products -> views.py

from django.shortcuts import render

def products(request):
    return render(request, 'products/products.html')

def product_details(request, id):
    return render(request, 'products/product_details.html')

In [ ]:
python manage.py runserver

### Designing the Database

The main target of this subject, as spoken before, is to get the details of some products from database and show them in HTML webpage (and vice versa). Meaning we will basically need to build the flow of data among all three layers of Django: Model, Template, View!

We'll start by making a model for our products just like the previous subject; migrate them to the database and then add some sample products to the database using Django Shell.

In [ ]:
# prodcuts -> models.py

from django.db import models

class Product(models.Model):
    name = models.CharField(default="N/A", max_length=100)
    price = models.IntegerField(default=0)
    is_active = models.BooleanField(default=True)

In [ ]:
python manage.py makemigrations
pyhton manage.py migrate
python manage.py shell

### Product List Page

Now we have completed our work with the Model layer. Time for View and Template. Suppose the body of `products.html` looks something like this:

In [ ]:
{% block body_refs %}

<h1>Hello, World!</h1>

<ul>
    <li><a href="">Item 1</a></li>
    <li><a href="">Item 2</a></li>
    <li><a href="">Item 3</a></li>
</ul>

{% endblock %}

The goal is to have thos `<li>` tags fetch their data from the database -> Product Model. Let's go back to views file and fetch the data there.

In [ ]:
def products(request):
    products = Product.objects.all()
    return render(
        request, 
        'products/products.html', 
        context={'products':products}
    )

You can see that everything we're using here is taught before, it's just the matter of mixing our knowledge into something practical.

Next up, alter the `products.html` file with some Python for loop to do the thing.

In [ ]:
{% block body_refs %}
<h1>Hello, World!</h1>

<ul>
    {% for prod in products %}
        <li>
            <a href="">
                {{ prod.name }}
            </a>
        </li>
    {% endfor %}
</ul>
{% endblock %}

Run the server once again to see the magic!

### Product Details Page

This next step is faily simple, we just need to build the connection of each `<a>` tag in product list to the relevant page of the desired product.

The key in doing so is the Primary Key in the database, which was id and was generated automatically after migration.

We'll start by adding the URL to the tag itself:

In [ ]:
<ul>
    {% for prod in products %}
        <li>
            <a href="{% url 'product_details' id=prod.id %}">
                {{ prod.name }}
            </a>
        </li>
    {% endfor %}
</ul>

This matter can also be done by using the `reverse()` method we saw earlier. We can alter the `Product` class in our models adding a method called `get_absolute_url(self)` which will eventually look something like this:

In [ ]:
from django.urls import reverse

def get_absolute_url(self):
    return reverse('product_details', args=[self.id]) # Args will fill out the <id> part of the url.

Then we move onto completing the `product_details()` view. If you have completed previous subjects cautiosly, you shouldn't have any problem understanding what's going on.

In [ ]:
def product_details(request, id):
    product = Product.objects.get(id=id)
    return render(
        request, 
        'products/product_details.html', 
        {'product':product}
    )

Finally, since styling isn't the target of this topic, we'll finish the work by a simple outline for `product_details.html` file.

In [ ]:
{% extends 'base.html' %}
{% load static %}

{% block head_refs %}
<link rel="stylesheet" href="{% static '/products/css/products.css' %}">
{% endblock %}

{% block title %}
{{ product.name }}
{% endblock %}

{% block body_refs %}
<h1>{{ product.name }}</h1>
<h2>{{ product.price }}</h2>
{% endblock %}

Restart the app if needed, and see what happens...

If you're a best-practice nerd like me and want to strictly follow the standards all the time, you gotta handle situations where the product does not exist in the database.

In [ ]:
from django.http import Http404
from django.shortcuts import get_object_or_404

def product_details(request, id):
    try:
        product = Product.objects.get(id=id)
    except:
        raise Http404
    
    # or simply
    product = get_object_or_404(Product, pk=id) # WHERE Primary Key = id
    
    return render(
        request, 
        'products/product_details.html', 
        {'product':product}
    )

There is one tiny detail which is worth mentioning here, that is the way the product is to be shown in URL. Right now, we simply feed the product's id into the `<id>` part in URL, which is fine but it's also possible to have a name shown in the URL instead of the product's id in the database.

To do so, we need to alter a few things. Navigate to `Product` model in `models.py` and add a new attribute, an instance of `SlugField()` class in `models` module. This class isn't anything magical, it just turns a word like "Hello World" into "hello-world", which is a valid notation for URLs.

At this point, you must very well know that you'd have to migrate once again to apply these changes to the database. What about previous data? We'll it's best to just clear everything out and generate new ones using ChatGPT same as before. The command below might be useful for clearing the already existing data in the database:

```bash
python manage.py flush
```

In [ ]:
from django.db import models

# Create your models here.
class Product(models.Model):
    name = models.CharField(default="N/A", max_length=100)
    price = models.IntegerField(default=0)
    is_active = models.BooleanField(default=True)
    slug = models.SlugField(default="", unique=True, db_index=True)

    def __str__(self):
        return f"{self.name}"

Note that the `db_index=True` performs an action called *Indexing* on this attribute of the database which will result in much faster data fetch and perfomance.

Then, it's time to change the `<id>` part to `<slug>` in URL path we made earlier in the `urls.py` file. This step, even though not necessary, it's best practice for clean code. Not cool to call something 'id' when it's the furthest thing from 'id'.

In [ ]:
from django.urls import path
from .EShop.products import views

urlpatterns = [
    path('', views.products, name='products'),
    path('product/<slug>', views.product_details, name='product_details'),
]

Next, we update the `product_details()` view in our views file to follow this new principles.

In [ ]:
def product_details(request, slug):
    try:
        product = Product.objects.get(slug=slug) # pay attention to this one!
    except:
        raise Http404
    return render(
        request, 
        'products/product_details.html', 
        {'product':product}
    )

Finally, we alter the `<a>` tag in our `products.html` file to point to the newly made field.

In [ ]:
<ul>
    {% for prod in products %}
        <li>
            <a href="{% url 'product_details' slug=prod.slug %}">
                {{ prod.name }}
            </a>
        </li>
    {% endfor %}
</ul>

Run the server and check out the difference!

**I have added a random number to the end of each slug in order to avoid similar slug vlaues as the records grow! Ever thaught what would happen if I hadn't?**

**Nerd Out**

Having multiple occurence of a slug value will eventually cause somthing to break. For example, you might recall that we handled the un available products in views like this:

In [ ]:
def product_details(request, slug):
    try:
        product = Product.objects.get(slug=slug)
    except:
        raise Http404
    return render(
        request, 
        'products/product_details.html', 
        {'product':product}
    )

Quote from official Django Documentation:

---

Note that there is a difference between using `get()`, and using `filter()` with a slice of \[0\]. If there are no results that match the query, `get()` will raise a `DoesNotExist` exception. This exception is an attribute of the model class that the query is being performed on - so in the code above, if there is no Entry object with a primary key of 1, Django will raise Entry.`DoesNotExist`.

Similarly, Django will complain if more than one item matches the `get()` query. In this case, it will raise `MultipleObjectsReturned`, which again is an attribute of the model class itself.

---

That's why we tend to use `get()` for this action, same as why it's prefred to set `unique=True` as we create the `SlugField()` attribute in the model.